# Chapter 14 – Deep Computer Vision Using Convolutional Neural Networks

This notebook contains all the sample code and solutions to the exercises in chapter 14.

1. Next, let’s set up a Conda environment:

```
conda create --name tf python=3.9
conda activate tf 
```

Make sure environment is activated for the rest of the installation

2. Install CUDA and cuDNN with conda (already done?)

3. Upgrade pip (unnecessary?)

4. Install TensorFlow
```
pip install "tensorflow<2.11" 
```

## Setup
This project requires Python 3.7 or above:

In [1]:
import sys

assert sys.version_info >= (3, 7)

It also requires Scikit-Learn ≥ 1.0.1:

In [2]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

And TensorFlow ≥ 2.8:

In [3]:
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

Current TensorFlow version:

In [4]:
version.parse(tf.__version__)

<Version('2.10.1')>

As we did in earlier chapters, let's define the default font sizes to make the figures prettier:

In [5]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

This chapter can be very slow without a GPU, so let's make sure there's one, or else issue a warning:

In [6]:
# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. Neural nets can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

In [7]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Convolutional Layers
## Implementing Convolutional Layers With Keras
Let's load two sample images, rescale their pixel values to 0-1, and center crop them to small 70×120 images:

In [8]:
from sklearn.datasets import load_sample_images
import tensorflow as tf

images = load_sample_images()["images"]

In [ ]:
images = tf.keras.layers.CenterCrop(height=70, width=120)(images)

In [ ]:
images = tf.keras.layers.Rescaling(scale=1 / 255)(images)

In [9]:
images.shape

TensorShape([2, 70, 120, 3])

: 

In [1]:
tf.random.set_seed(42)  # extra code – ensures reproducibility
conv_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=7)
fmaps = conv_layer(images)

NameError: name 'tf' is not defined